# <u>DIU EIL UPF Bloc 5 :</u> Algorithmique Avancée

## <u>Annexe du Projet Taquin 3x3 :</u> Algorithmes de Recherche pour le Jeu du Taquin

Document d'accompagnement pour le Projet Taquin 3x3

<u>Auteurs :</u> Didier CHIU, Nicolas VAZ-PINTO<br> 
<u>Contact :</u> d.chiu@clm.ddec.pf, n.vazpinto@clm.ddec.pf<br> 
<u>Etablissement :</u> Lycée La Mennais (Papeete TAHITI)<br>
<u>Date de publication :</u> 20/04/2020<br>
<u>Niveau :</u> Terminale NSI<br>

# Sommaire <a id="sommaire"></a>

[**I. Préambule**](#preambule)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[I.A Intentions majeures](#intentions)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[I.B Représentation du taquin](#branching-factor)<br> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[I.C Chaos](#chaos)<br><br> 
[**II. BFS**](#bfs)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[II.A Principe](#bfs-1)<br> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[II.B Preuves](#bfs-2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[II.C Complexité](#bfs-3)<br><br> 
[**III. DFS**](#dfs)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[III.A Principe](#dfs-1)<br> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[III.B Preuves](#dfs-2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[III.C Complexité](#dfs-3)<br><br>
[**IV. DLS**](#dls)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[IV.A Principe](#dls-1)<br> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[IV.B Preuves](#dls-2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[IV.C Complexité](#dls-3)<br><br>
[**V. IDS**](#ids)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[V.A Principe](#ids-1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[V.B Preuves](#ids-2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[V.C Complexité](#ids-3)<br><br>
[**VI. IDA***](#ida)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[VI.A Principe](#ida-1)<br> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[VI.B Preuves](#ida-2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[VI.C Complexité](#ida-3)<br><br>
[**VII. TLDR**](#tldr)<br><br>
[**Licence**](#licence)

# I. Préambule <a id="preambule"></a>

## I.A Intentions majeures  <a id="intentions"></a>

Ce Notebook a pour but d'alléger le Notebook du Projet en fournissant les descriptions des algorithmes de recherche, leurs preuves (correction, terminaison et optimalité) ainsi qu'une étude de leurs complexités.<br>

Il est également conçu comme préambule à la réalisation de la fiche de l'enseignant pour l'encadrement du projet.<br>
Il a été rédigé avec à l'esprit l'envie d'être le plus complet possible et de fournir rapidement les clefs à tout enseignant désireux de reprendre ou de s'insipirer de notre travail

<u>Rappel :</u> Projet réalisé dans le cadre de l'évaluation encadrée par <a href=mailto:jean-marc.vincent@univ-grenoble-alpes.fr>Jean-Marc VINCENT</a> (Université Grenoble-Alpes).<br>
Ce projet est à destination des élèves de Terminale NSI et est centré autour du jeu du taquin 3x3.<br>

### Formulation du projet 

- **Enoncé :** L'objectif de ce projet est de réaliser une interface Web permettant de jouer au jeu du taquin
    - L'interface web devra communiquer avec un module python pour effectuer certains calculs
    - En effet, l'interface Web devra proposer :
        - un jeu sur un taquin 3x3
        - un bouton &laquo; aide &raquo; qui propose au joueur un &laquo; coup suivant &raquo; le rapprochant de la solution
        - un bouton &laquo; abandon &raquo; qui déclencher une résolution automatique et animée du taquin
    
- **Cahier des charges :**    
    - Le travail est à faire en binômes ou trinômes
    - Le travail est à rendre 3 semaines après le lancement du projet on compte 2 semaines de classe et 1 semaine de vacances
    - L’interface HTML doit posséder un bouton d’aide et un système d’affichage qui permettra d’indiquer à l’utilisateur quel est le meilleur coup suivant. On implémentera également un bouton d'abandon qui arrêtera le jeu et lancera une animation montrant la résolution du taquin
    - La mise en place d’un server Flask pour récupérer les données depuis la page HTML et transmettre le meilleur coup calculé par l’algorithme dans le module python
    - Le module python doit utiliser un algorithme présenté dans les ressources pour trouver le meilleur coup à partir de l’état actuel de la page client
    - Le code source doit être commenté
    
- **Langages et bibliothèques :**    
    - HTML / CSS
    - JavaScript
        - JQuery (librairie de JavaScript)
        - GSAP (librairie d'animation)
    - Python
        - Flask (web framework càd client/server local) 
        - Jinja2 (templating language càd langage de création de modèles)

[**Retour au sommaire**](#sommaire)

## I.B Représentation du taquin <a id="branching-factor"></a>

### Ressources
- <a href="https://cs.stackexchange.com/questions/39534/how-to-find-the-branch-factor-of-8-puzzle">Average branching factor</a>

Un moyen simple de représenter un état du taquin est d'utiliser une chaîne de caractères comme par exemple &laquo; 012345678 &raquo; pour l'état initial.<br>
A partir d'un état donné, un état au coup suivant sera donc la chaîne précédente où deux caractères auront échangé leurs positions dans la chaîne.<br>
Rechercher une solution pour ranger le taquin revient donc à parcourir un graphe orienté où chaque sommet correspond à un état et où chaque arc correspond au déplacement d'une tuile dans l'une des quatres directions haut, bas, gauche ou droite.<br>
On appellera &laquo; successeur &raquo; d'un état $e$ l'un des états accessible à partir de l'un de ces déplacements.

**A propos du nombre de successeurs d'un état :** <a id="branching-factor-1"></a>
- On note $b$ le nombre de successeurs d'un état ($b$ pour &laquo; branching factor &raquo;)
- On a $b \leq 4$ car tout état possède au plus 4 successeurs
- En effet :
    - Si la case vide est au milieu (1 état possible) : 4 successeurs
    - Si la case vide est dans un coin (4 états possibles) : 2 successeurs
    - Si la case vide est sur un milieu (4 états possibles) : 3 successeurs
    - En moyenne : $\dfrac{1 \times 4 + 4 \times 2 + 4 \times 3}{4+4+1}=\dfrac{24}{9} \simeq 2,66...$
- Si de plus, on élimine de la liste des successeurs un état déjà visité (les mouvements comme Haut puis Bas ou Gauche puis Droite nous ramènent à un état déjà visité), on obtient :
    - Si la case vide est au milieu (1 état possible) : 3 successeurs
    - Si la case vide est dans un coin (4 états possibles) : 1 successeur
    - Si la case vide est sur un milieu (4 états possibles) : 2 successeurs 
    - En moyenne : $\dfrac{1 \times 3 + 4 \times 1 + 4 \times 2}{4+4+1}=\dfrac{15}{9} \simeq 1,66...$ <br>
    
**Profondeur maximale pour la visiter tous les états :** <a id="branching-factor-2"></a>
- On peut estimer la profondeur $p$ pour laquelle se trouve un état solution pour une majorité des états de départ :
    - Il y a $\dfrac{9!}{2}= 181 440$ états de départ solvables
    - On cherche $d$ tel que $b^d > \dfrac{9!}{2}$ (c'est-à-dire à quelle profondeur aura-t-on visités tous les états)
    - On obtient $d \cdot \ln(b) > \ln \left(\dfrac{9!}{2} \right) = \ln(9!)-\ln(2)$
    - Donc $d > \dfrac{\ln(9!)-\ln(2)}{\ln(b)}$
- Ainsi :

|$b$|$p>$|
|:--:|:--:|
|4|8,73|
|3|11,02|
|2,66|12,37|
|2|17,46|
|1,66|23,70|
|1,5|29,86|

- En pratique on trouve une profondeur maximale de 28

**Conséquences :**
- Pour espérer atteindre un état solution, il faut que les algorithmes de recherche puissent descendre à une profondeur suffisante
- Pour diminuer la valeur de $b$, il faut être capable de stocker l'ensemble des états déjà atteints afin de comparer un état nouvellement découvert avec tous les éléments de cet ensemble
- Cette opération peut s'avérer coûteuse à mesure que l'on descend en profondeur et que le nombre d'états déjà atteints augmente (voir les études de complexité)
- Pour des considérations liées à la représentation du graphe d'états, les exemples ne montreront que des états réellement &laquo; nouveaux &raquo;

<img src="images/taquingraphetat.png" style="width: 60%"/>

[**Retour au sommaire**](#sommaire)

## I.C Chaos <a id="chaos"></a>
>&laquo; Le chaos n'est qu'un principe méconnu de l'ordre &raquo;

### Ressources :
- <a href="https://www.youtube.com/watch?v=-3IsCOJieCc&t=312s">Vidéo Youtube Chaîne : Micmaths</a>
- <a href="http://www.jeuxmath.ch/solutions/64-taquin.pdf">Une approche plus classique sur la validité</a>

On commence le jeu du taquin avec une configuration de départ où les tuiles sont mélangées.<br>
Une façon d'obtenir cette configuration de départ serait de mélanger aléatoirement les tuiles à partir du taquin rangé.<br>
Cette approche &laquo; naturelle &raquo; (car c'est ce qui se produit lorsqu'on déballe le taquin de son emballage : on bouge les tuiles un peu hasard quand même !) conduit toujours à une configuration acceptable (qui conduit à un état solution du taquin).<br>

Plusieurs approches peuvent alors être envisagées par les élèves :<br>
**1)** Introduire une mesure du dérangement (voir vidéo Micmaths) : on montre que cette quantité doit être paire pour que le taquin soit soluble. Effectuer alors un mélange aléatoire tant que cette quantité est impaire.

**2)** Partir du taquin résolu et ne faire que des déplacements de tuiles, choisis aléatoirement parmi Haut, Bas, Gauche, Droit.<br>
Cette approche ne nécessite pas l'introduction d'une mesure du dérangement. De plus, si on stocke les déplacements effectués, on sait reconstruire l'enchaînement des mouvements pour retrouver l'état solution du taquin, ce qui peut être appréciable pour faire des tests...

**Remarques**
- La première approche :
    - permet de ne pas se préoccuper de la représentation du taquin et de ses états suivants (classes) mais il faut que les élèves maîtrisent les va-et-vient entre chaîne de caractères et liste
    - donne un taquin &laquo; très &raquo; mélangé, ce qui peut donner des profondeurs pour l'état solution inadapté comme nous le verrons dans la partie de complexité
- La seconde approche :
    - nécessite d'avoir déjà codé les mouvements autorisés dans le taquin (utilisation des classes)
    - donne un taquin &laquo; peu &raquo; mélangé puisqu'elle peut produire des successions de mouvements telles que {H, B}, {B, H}, {G, D} ou {D, G} (qui ne mélangent pas réellement le taquin ce qui permet donc de faire rapidement et facilement des essais)
    
**Take home message :**
- Le choix de la &laquo; meilleure &raquo; approche est laissée à la discrétion des enseignants qui réutiliseront ce travail
- Ce pré-conditionnement du graphe d'états du taquin nous assure de travailler avec un graphe où le chemin menant à l'état solution existe et est fini (long peut-être, mais fini)

[**Retour au sommaire**](#sommaire)

In [6]:
# Première approche : mélange aléatoire tant que la mesure du dérangement n'est pas paire

# Le module random dispose d'une fonction shuffle qui mélange les éléments d'une liste
# Il suffit de créer une chaîne à partir des éléments de la liste mélangée
import random

def shufflelist(s):
    L = list(s)
    random.shuffle(L)
    result = ''.join(L)
    return result

# Mesure le dérangement d'un état du taquin
# Un état est représenté par une chaîne de caractères
def levelofchaos(etat):
    level = 0
    for i in range(len(etat)):
        for j in range(i+1, len(etat)):
            #print("couple", etat[i], etat[j])
            if etat[i] > etat[j]:
                #print("elements mal rangés")
                level += 1
    return level
    
# solvable : 125374680  
# non solvable : 230518674
print("Niveau de dérangement pour 125374680", levelofchaos('125374680'))
print("Niveau de dérangement pour 230518674", levelofchaos('230518674'))

# Mélange aléatoirement un taquin
# statetaquin est une chaîne représentant l'état du taquin
def melanger(statetaquin):
    # on crée la liste correspondante
    L = []
    for i in range(0, len(statetaquin)):
        L.append(statetaquin[i])
    randstate = shufflelist(L)
    
    nbpass = 0
    # on mélange la chaîne ainsi produite tant que la mesure du dérangement n'est pas paire    
    while (levelofchaos(randstate) % 2 != 0):
        nbpass += 1
        randstate = shufflelist(L)
    print("Nb de mélanges aléatoires effectués", nbpass)
    # on renvoie la chaîne mélangée pour une configuration acceptable
    # on pourrait aussi renvoyer une liste contenant les éléments de la chaîne
    return randstate

newstate = melanger('230518674')
print("Un nouvel état acceptable à partir de 230518674 est", newstate, "mesure du dérangement", levelofchaos(newstate))

Niveau de dérangement pour 125374680 12
Niveau de dérangement pour 230518674 11
Nb de mélanges aléatoires effectués 2
Un nouvel état acceptable à partir de 230518674 est 847130265 mesure du dérangement 22


*Deuxième approche :  partir du taquin initial et ne faire que des déplacements de tuiles, choisis aléatoirement parmi Haut, Bas, Gauche, Droit.*

Extrait du code de la classe Taquin :
```python
def melanger(self, nbpass):
    for i in range (0, nbpass):
        move = randint(1,4)
        if move == 1:
            self.haut()
        elif move == 2:
            self.bas()
        elif move == 3:
            self.droite()
        elif move == 4:
            self.gauche()
```

La variable nbpass correspond donc aux nombre de mouvements effectués pour produire un état initial du taquin<br>
Ce n'est pas la profondeur à laquelle se trouve l'état solution par rapport à l'état initial puisqu'on peut produire (même aléatoirement) des successions de mouvements telles que {H, B}, {B, H}, {G, D} ou {D, G} (qui ne mélangent pas réellement le taquin).

[**Retour au sommaire**](#sommaire)

# II. BFS (Breadth First Search) <a id="bfs"></a>

### Ressources :
- <a href="http://math.univ-lyon1.fr/irem/IMG/pdf/parcours.pdf">Des rappels de cours page 17/123 (slide 11/97)</a>
- <a href="https://fr.wikipedia.org/wiki/Algorithme_de_parcours_en_largeur">Wikipedia FR</a> 
- <a href="https://www.youtube.com/watch?v=OYwF580EDWY">Une vidéo sur BFS</a>
- <a href="http://people.irisa.fr/Francois.Schwarzentruber/agreg_algo_2015/parcourslargeur_dijkstra.pdf">Corrigé d'AGREG 2015</a>

## II.A Principe <a id="bfs-1"></a>
- On utilise un parcours en largeur (utilisation d'une file) pour explorer les états du graphe à une profondeur $p$ avant de passer aux états suivants (profondeur $p+1$) 
- Quand on trouve un état solution, on renvoie la profondeur à laquelle il se trouve

**Rappel : Parcours en largeur (PEL) dans un graphe :**

<u>Principe général :</u> tous les sommets à un certaine distance $d$ sont rencontrés avant le premier sommet à distance supérieure à $d$
- Parcourir les sommets adjacents au sommet de départ et les stocker dans une file
- Pour chaque sommet défilé, parcourir ses adjacents et les enfiler
- Recommencer jusqu'à ce que la file soit vide

**Pseudo code du PEL :**
```
ParcoursLargeur(Graphe G, Sommet s):
    f = CreerFile();
    f.enfiler(s);
    marquer(s);
        Tant que la file est non vide
            s = f.defiler();
            afficher(s);
            Pour tout voisin t de s dans G
                Si t non marqué
                    f.enfiler(t);
                    marquer(t);
```

**Adaptation du PEL pour le graphe d'états du taquin :**
- A partir de l'état initial du taquin $e_0$, on enfile les états suivants de $e_0$
- Pour chaque état dans la file, on le traite (càd on le défile) en enfilant ses états suivants
- Recommencer jusqu'à ce que la file soit vide

**Illustration du PEL pour le graphe d'états du taquin sur deux niveaux :**

<img src="images/bfs.gif" style="width: 60%"/>

**Adaptation du PEL pour chercher un état solution du taquin (BFS) :** 
- On initialize la file avec l'état $e_0$
- Tant que la file n'est pas vide, on traite la file, c'est-à-dire :
    - Pour chaque état $e$ à la profondeur $p$, on effectue le traitement :
        - Si cet état est gagnant, on sort du Tant que
        - Ensuite on enfile les états suivants de $e$ (ceux qui se trouvent à la profondeur $p+1$)
        - On retire $e$ de la file (défiler)
    - On passe à la profondeur $p+1$

**Pseudo-code :**
```
BFS(e0)                                   //e0 désigne un état de départ du taquin
	A = {e0}                              //A désigne la file
	p = 0
	Tant que A n’est pas vide
		B = {}
		Pour tout x appartenant à  A      // x désigne un état du taquin
			Si x est la solution alors
				renvoyer p
			B = s(x) U B                  // s(x) désigne l'ensemble des états suivant du taquin à l'état x
            FinSi
        FinPour
		A = B
		p = p+1
```

**Code python :**
```python
def bfs(etatdepart):
    """
    Fonction de recherche qui part d'un état initial du taquin et effectue un parcours en largeur jusqu'à ce qu'elle trouve l'état gagnant. A ce moment elle renvoie la profondeur de la solution trouvée. 
    Principe : Elle teste chaque état d'une profondeur donnée avant de passer à la profondeur suivante dans le cas où elle ne trouve pas la combinaison gagnante à cette profondeur.
    Entrée : un état du taquin
	Sortie : profondeur à laquelle la solution aura été trouvée
    """
    
    # état de départ pour lancer la recherche avec parcours en largeur
    A = [etatdepart]
        
    # profondeur de départ
    p = 0
    while A != []:
		# initialisation de la liste des états suivants
        B = []
        # boucle qui teste un par un les états de profondeur p
        for i in range(0, len(A)):
            # On vérifie si l'état est gagnant
            taquindetravail = Taquin(A[i])
            if taquindetravail.estGagnant():
                # renvoie la profondeur de la solution
                return p
                        
            # On rajoute les états suivants pour chaque élément de la file
            for i in taquindetravail.suivants():
                # ajoute chaque état suivant à la liste qui sera testée à la profondeur suivante
                B.append(i)
        A = B
        # pour passer à la profondeur suivante
        p = p + 1 
```

[**Retour au sommaire**](#sommaire)

## II.B Preuves <a id="bfs-2"></a>

### II.B.1 Terminaison

**Rappel du pseudo-code :**
```
BFS(e0)                                   //e0 désigne un état de départ du taquin
	A = {e0}                              //A désigne la file
	p = 0
	Tant que A n’est pas vide
		B = {}
		Pour tout x appartenant à  A      // x désigne un état du taquin
			Si x est la solution alors
				renvoyer p
			B = s(x) U B                  // s(x) désigne l'ensemble des états suivant du taquin à l'état x
            FinSi
        FinPour
		A = B
		p = p+1
```

- Les ensembles A et B ne contiennent que des états atteignables depuis $e_0$ :
    - En effet, A est initialisé à $e_0$ et est remplacé par B à la fin de la boucle Pour
    - Or B est modifié (mis à jour) à chaque passage dans la boucle Tant que en lui rajoutant les ensembles $s(x)$ où $x$ est dans A
    - Ainsi, s’il n’existe pas de solution atteignable depuis $e_0$, l'ensemble A ne contiendra aucun état atteignable, donc l’algorithme ne renverra pas VRAI (ce qui n'est pas notre cas)

- Réciproquement, supposons qu’un état gagnant $e_p$ (à la profondeur $p$) est atteignable depuis $e_0$ :
    - Alors il existe un chemin $e_0, ..., e_p$ 
    - Alors par construction de l’ensemble B, B (puis A) contiendra à la fin du k-ème passage dans la boucle Tant que l’élément $e_k$ (ou aura déjà renvoyé la valeur de la profondeur avant) 
    - En effet, si $e_k$ est dans A après le $k^{ème}$ passage, alors on ajoute $s(e_k)$ à B, c'est-à-dire qu'on aura $e_{k+1}$ dans B (puis dans A) après la $(k + 1)^{ème}$ itération
    - Ceci montre que $e_p \in$ A
    - Finalement, si l’algorithme n’a pas renvoyé la valeur de la profondeur avant, il s’arrêtera au $p^{ème}$ passage

### II.B.2 Correction 

### Ressources
- <a href="https://perso.liris.cnrs.fr/christine.solnon/supportAlgoGraphes.pdf">Preuve de correction du PEL page 16/49</a>
- [Initialisation du taquin pour obtenir une configuration acceptable](#chaos)

La correction de l'algorithme BFS découle de l'existence d'un état solution (obtenu par le pré-traitement avec une fonction de mélange qui nous assure la solvabilité du taquin) et de la correction du PEL (qui permet de parcourir tous les états dans notre graphe)

### II.B.C Optimalité 

### Ressources
- <a href="https://cs.stackexchange.com/questions/16758/artificial-intelligence-condition-for-bfs-being-optimal">Un forum de discussion en anglais</a>
- <a href="http://informatique.umons.ac.be/algo/astar.pdf">Des slides en français (voir slide 42/75)</a>

En général, l'algorithme BFS n'est pas optimal. Cependant dans notre cas, l'optimalité de l'algorithme BFS est assurée par :
- le coût de passage d'une profondeur à la profondeur suivante est une fonction non décroissante 
- les coûts de passage sont constants

Une autre formulation pour les élèves consisterait à dire que :
- la recherche est complète (càd on visite tous les états)
- la profondeur est augmentée au fur et à mesure et le coût est identique pour tout état vers le suivant
- donc le premier état solution trouvé est celui de plus courte distance avec l'origine

[**Retour au sommaire**](#sommaire)

## II.C Complexité<a id="bfs-3"></a>
- Complexité en temps : nombre d'états qu'il faut visiter (ajouter à la file) pour atteindre l'état solution
- Complexité en espace : nombre maximum d'états à conserver pour atteindre l'état solution
- Complexité expérimentale : mesure du temps que le programme met pour se terminer

### Ressources 
- <a href="https://perso.liris.cnrs.fr/christine.solnon/supportAlgoGraphes.pdf">Complexité du PEL page 15/49</a>

**Rappel du pseudo-code :**
```
BFS(e0)                                   //e0 désigne un état de départ du taquin
	A = {e0}                              //A désigne la file
	p = 0
	Tant que A n’est pas vide
		B = {}
		Pour tout x appartenant à  A      // x désigne un état du taquin
			Si x est la solution alors
				renvoyer p
			B = s(x) U B                  // s(x) désigne l'ensemble des états suivant du taquin à l'état x
            FinSi
        FinPour
		A = B
		p = p+1
```

### II.C.1 Complexité en Temps

Soit $V$ l'ensemble des états et $E$ l'ensemble des arcs du graphe d'état du taquin.<br>
Alors, avec $|V|$ est le nombre d'états du graphe et $|E|$ le nombre d'arcs.

**Preuve :**
- Chaque état accessible depuis $e_0$ est mis au plus une fois dans la file A
    - En effet, à chaque fois qu'un état entre dans la file, il est défilé une fois qu'on a enfilé ses états suivants
- A chaque passage dans la boucle while, il y a exactement un état qui est défilé
    - Cette boucle sera donc exécutée au plus $|V|$ fois
- A chaque fois qu'un état est défilé, la boucle Pour parcourt les états suivants de celui que l'on vient de défiler
    - Le test (Si x est la solution) et la mise à jour de la file ( B = s(x) U B ) ne seront effectués donc qu'au plus une fois par arc
- Donc, la complexité en espace de l'algorithme BFS est en $O(|V|+|E|)$

**Remarques :**
- Une complexité en $O(|V|+|E|)$ signifie que la complexité sera en $O(|V|)$ si le nombre d'états domine le nombre d'arcs et sera en $O(|E|)$ sinon
- Dans notre cas, $|V|$ est équivalent à $|E|$
   
**Estimation de la complexité :**   
- On note $d$ la profondeur à laquelle se trouve un état solution
- $e_0$ possède au plus $b$ successeurs (si la case vide est au milieu de la grille) et cela reste valable pour chaque état suivant : $|V| \leq b^d$
- D'après le point précédent, chaque état $e$ à la profondeur $p$ engendre au plus $b$ successeurs à la profondeur $p+1$
- D'où $|E| \leq b^d$
- Donc la complexité en temps de l'algorithme BFS étant en $O(|V|+|E|)$ on obtient $O(b^d+b^d)$ càd en $O(b^d)$

**Take home message :**
- La complexité en temps est exponentielle, ce qui est bien mauvais quand $d$ est &laquo; grand &raquo;, et gagner un facteur 2 sur $b$ n'est pas l'objectif de notre travail algorithmique 

### II.C.2 Complexité en Espace

L'algorithme BFS garde en mémoire les successeurs de tous les états d'une même profondeur avant de passer à la profondeur suivante.

- On considère un état à la profondeur $p$
- Pour passer à la profondeur $p+1$ il faut générer au plus $b$ successeurs (et les mettre dans la file)
- Le nombre d'états ayant été mis dans la file pour atteindre la profondeur $d$ est donc majoré par $1+b+b^2+b^3+...+b^d = \displaystyle{\sum_{k=0}^{d}b^k}$
- Donc l'espace mémoire nécessaire est en $O(b^d)$
- Donc la complexité en espace est exponentielle en $d$

**Moralité :**
- La complexité en espace est elle aussi exponentielle en $d$, ce qui semble bien mauvais
- On retiendra que l'algorithme BFS est gourmand en ressources (temps et espace) mais reste optimal

### II.C.3 Complexité expérimentale

- **Protocole de Test**
    - Spécification de la machine de test : 
        - Processeur Intel Core i7-8550U CPU @ 1,8GHz 
        - Ram 8Go  
    - Lancer 100 fois l'algorithme BFS et mesurer le temps d'exécution avec le module time
    - Pour des profondeurs plus importantes, on réduira la quantité de fois où on lance BFS
    - La liste des taquins de départ utilisée est disponible dans un document annexe <br><br>
    
- **Code python :**

```python
# Test 100 BFS
def testBFS(state, n):
    moy = 0
    for i in range(n):
        debut = time.time()
        profsol = bfs(state)
        bfs(state)
        fin = time.time() - debut
        moy += fin
    return moy/n

print("moyenne 100 BFS", testBFS(etat, 100))
```

- **Tableau récapitulatif :**

| Profondeur | Temps d'exécution (ordre de grandeur) |
|:---:|:---:|
|2|0,06 ms|
|3|0,19 ms|
|4|0,31 ms|
|5|1,55 ms|
|6|1,75 ms|
|7|15,5 ms|
|8|20 à 60 ms|
|9|70 à 150 ms|
|10|125 à 400 ms|
|11|625 à 1150 ms|
|12|1 à 2 secondes|
|13|8 à 10 secondes|
|14|11 à 18 secondes|
|15|t > 80 s ou plante|
|16 et plus|plante|

[**Retour au sommaire**](#sommaire)

# III. DFS (Depth First Search) <a id="dfs"></a>

### Ressources :
- <a href="http://math.univ-lyon1.fr/irem/IMG/pdf/parcours.pdf">Des rappels de cours page 65/123 (slide 47/97)</a>
- <a href="https://fr.wikipedia.org/wiki/Algorithme_de_parcours_en_profondeur">Wikipedia FR</a>
- <a href="https://www.youtube.com/watch?v=p4YM7iy_YfE">Une vidéo sur DFS</a>

## III.A Principe <a id="dfs-1"></a>
- On utilise un parcours en profondeur (utilisation d'une pile) pour explorer les états du graphe
- Quand on trouve un état solution, on construit la liste des états nécessaires pour atteindre cette solution

**Rappel : Parcours en profondeur (PEP) dans un graphe :**

<u>Principe général :</u> les sommets accessibles depuis le sommet découvert le plus récemment sont explorés avant de continuer le parcours

- Version avec ue pile :
    - Empiler le sommet de départ
    - Si le sommet de départ présente des successeurs qui ne sont pas encore présents dans la pile, ni déjà passés dans la pile
        - Sélectionner l'un de ces successeurs et l'empiler
        - Sinon dépiler
    - Recommencer jusqu'à ce que la pile soit vide
- Version récursive :
    - Explorer les successeurs du sommet initial
    - Pour tout successeur qui en possède encore, relancer l'exploration à la profondeur suivante
    
**Pseudo code (PEP implémentation récursive) :**
```
explorer(graphe G, sommet s)
      marquer le sommet s
      afficher(s)
      Pour tout sommet t fils du sommet s
            Si t n'est pas marqué alors
                   explorer(G, t);
                   
//Le parcours en profondeur d'un graphe G est alors :

parcoursProfondeur(graphe G)
      Pour tout sommet s du graphe G
            Si s n'est pas marqué alors
                   explorer(G, s)
```
    
**Adaptation du PEP pour le graphe d'états du taquin :**
- Explorer les états suivants de l'état initial $e_0$ 
- Pour tout état suivant, appel récursif du PEP en prenant pour nouvel état initial l'état suivant considéré

**Illustration du PEP pour le graphe d'états du taquin sur deux niveaux :**

<img src="images/dfs.gif" style="width: 60%"/>

**Adaptation du PEP pour chercher un état solution du taquin (DFS) :** 

- La pile sera implémentée par une fonction récursive (on n'a plus à décrire explicitement la pile)
- On initialize la pile avec l'état $e_0$
- De plus, on ne s'attardera pas à garder en mémoire les états déjà visités ou déjà passés dans la pile 
[**Voir ici**](#branching-factor)

**Pseudo code (implémentation récursive) :**
```
DFS(e0, p0, chemin)                             //e0 désigne un état de départ du taquin  
    Si e0 est gagnant
        renvoyer Vrai
    FinSi
    
    L = liste_des_etats_suivants(e0)            //L désigne la liste des états suivants de e0
    Pour tout x de L
        Si DFS(x, p0+1, chemin) a renvoyé Vrai
            ajouter x à chemin                  //le chemin vers l'état solution se contruit
            renvoyer Vrai
        FinSi
    FinPour
    
    renvoyer Faux
```

**Code python :**

```python
def dfs(etatdepart, profdepart, chemin):
    """
    Fonction parcours le graphe d'états en profondeur et qui construit la liste des états entre l'état de départ et l'état solution
    Entrées :
        - etatdepart : état de départ
        - profdepart : profondeur initiale
        - chemin : chemin (liste des états successifs) d'accès à l'état gagnant
    Sorties : 
    """
    
    # vérification si l'etat testé est gagnant ou non
    taquindetravail = Taquin(etatdepart)
    if taquindetravail.estGagnant():
        return True
        
    # parcours récursif des états suivants
    # appel récursif de la fonction de test pour descendre en profondeurfor k in listeEtatsSuivants:
    for etatsuivant in taquindetravail.suivants():
        if dfs(etatsuivant, profdepart + 1, chemin) == True:
            #stockage du chemin d'accès en mémoire et retour
            chemin.append(etatsuivant)
            return True
        
    return False
```

[**Retour au sommaire**](#sommaire)

## III.B Preuves <a id="dfs-2"></a>

### III.B.1 Terminaison

En général, l'algorithme DFS ne termine pas :
- Si la graphe est infini (ce qui n'est pas le cas ici)
- Si l'algorithme tombe dans un cycle (ce qui n'est pas possible si on garde en mémoire les états déjà rencontrés où passés par la pile)

### III.B.2 Correction

### Ressources
- <a href="https://www.irif.fr/~francoisl/DIVERS/slides-parcours.pdf">Preuve de correction du PEP à partir du slide 68/93</a>
- [Initialisation du taquin pour obtenir une configuration acceptable](#chaos)

La correction de l'algorithme DFS découle de l'existence d'un état solution (obtenu par le pré-traitement avec une fonction de mélange qui nous assure la solvabilité du taquin) et de la correction du PEP (qui permet de parcourir tous les états dans notre graphe)

### III.B.3 Optimalité

### Ressources
- <a href="https://stackoverflow.com/questions/21264053/why-dfs-is-not-optimal-but-bfs-is-optimal">Un forum de discussions en anglais</a>

En général, l'algorithme DFS n'est pas optimal :
- On suppose que l'algorithme DFS explore d'abord le successeur &laquo; gauche &raquo; de l'état initial
- Il se peut que l'algorithme DFS trouve alors une solution à la profondeur $m$ alors que la solution optimale se trouve à une profondeur $d < m$ dans l'un des successeurs &laquo; à droite &raquo; de l'état initial 

[**Retour au sommaire**](#sommaire)

## III.C Complexité <a id="dfs-3"></a>
- Complexité en temps : nombre d'états qu'il faut visiter pour atteindre l'état solution
- Complexité en espace : nombre maximum d'états à conserver pour atteindre l'état solution
- Complexité expérimentale : mesure du temps que le programme met pour se terminer

### Ressources
- <a href="https://perso.liris.cnrs.fr/christine.solnon/supportAlgoGraphes.pdf">Complexité en temps du PEP page 17/49</a>
- <a href="http://jul.andre.free.fr/Cavaliers/cours2_RechAveugle.pdf">DFS (slides 49 à 73) et DLS (slides 74 à 76) dans un cours de l'université de Nancy</a>

<u>**Rappel :**</u> **Pseudo-code :**
```
DFS(e0, p0, chemin)                             //e0 désigne un état de départ du taquin  
    Si e0 est gagnant
        renvoyer Vrai
    FinSi
    
    L = liste_des_etats_suivants(e0)            //L désigne la liste des états suivants de e0
    Pour tout x de L
        Si DFS(x, p0+1, chemin) a renvoyé Vrai
            ajouter x à chemin                  //le chemin vers l'état solution se contruit
            renvoyer Vrai
        FinSi
    FinPour
    
    renvoyer Faux
```

### III.C.1 Complexité en Temps

Soit $V$ l'ensemble des états et $E$ l'ensemble des arcs du graphe d'état du taquin.<br>
Alors, avec $|V|$ est le nombre d'état du graphe et $|E|$ le nombre d'arcs : 

**Preuve pour la version itérative :**
- Chaque état accessible depuis $e_0$ est mis puis enlevé au plus une fois dans la pile (comme dans BFS)

- A chaque passage dans la boucle while, il y a exactement un état qui est dépilé
    - Soit un état est empilé (si l'état considéré a encore des successeurs)
    - Soit un état est dépilé (si l'état considéré n'a plus de successeurs)
    - Cette boucle sera donc exécutée au plus $2|V|$ fois
- A chaque fois qu'un état est dépilé, la boucle Pour parcourt les états suivants de celui que l'on vient de dépiler
    - Le test (Si x est la solution) et la mise à jour de la file (B=s(x) U B) ne seront effectués donc qu'au plus une fois par arc
- Donc, la complexité de l'algorithme DFS est en $O(|V|+|E|)$

**Preuve pour la version récursive :**
- La version récursive de DFS est appelée au plus $|V|$ fois
- Chaque arête est considérée au plus une fois dans la boucle for
- Donc la complexité est en $O(|V| + |E|)$

**Estimation de la complexité :**
- On note $m$ la profondeur maximum d'un état dans le graphe
- $e_0$ possède au plus $b$ successeurs et cela reste valable pour chaque état suivant : $|V| \leq b^m$ 
- Par suite, chaque état $e$ à la profondeur $p$ engendre au plus $b$ successeurs à la profondeur $p+1$
- D'où $|E| \leq b^m$
- Donc la complexité en temps de l'algorithme DFS étant en $O(|V|+|E|)$ on obtient $O(b^m+b^m)$ càd en $O(b^m)$

**Take home message :**
- Avec $m$ est très supérieur à $d$, cette complexité exponentielle est affreuse !

### III.C.2 Complexité en Espace

L'algorithme DFS garde en mémoire un nombre d'états $\leq b$ par niveau de profondeur

- Pour arriver à un état à la profondeur $p < m$ il faut a donc fallut stocker au plus $b \times p$ états ($b$ successeurs par niveau de profondeur)
- Ainsi, il faut stocker $b \times (m-1)$ pour arriver à la profondeur $m-1$ et on va stocker $b$ états de plus pour descendre à la profondeur $m$
- D'où $b \times (m-1) + b = b \times m$ états sont stockés en mémoire pour accéder à la profondeur $m$
- Donc la complextié en espace est en $O(b \times m)$

**Moralité :**

- La complexité en espace est linéaire en $m$, ce qui est bien plus intéressant que pour BFS (exponentielle en $d$ la profondeur de la solution)
- On retiendra que l'algorithme DFS est plus efficace dans la gestion de la mémoire mais en paye le prix en perdant l'optimalité et nécessite des précautions pour garantir sa complétude par rapport à l'algorithme BFS.

[**Retour au sommaire**](#sommaire)

# IV. DLS (Depth-Limited Search) <a id="dls"></a>

Egalement connu sour le nom de BDFS (Bounded Depth First Search)

### Ressources
- <a href="https://enacademic.com/dic.nsf/enwiki/446185">Une fiche sur DLS en anglais</a>
- <a href="https://www.youtube.com/watch?v=2af6E3nMWVM"> Un vidéo Youtube sur DLS</a> 

### IV.A Principe <a id="dls-1"></a>
- On utilise adapte [l'algorithme DFS précédent](#dfs) pour explorer les états du graphe en descendant jusqu'à une profondeur maximale (profmax)
- Cette profondeur est donnée par [l'algorithme BFS](#bfs) ou peut être [fixée arbitrairement au départ](#branching-factor-2)
- Si on ne trouve pas d'état solution jusqu'à la profondeur profmax, on s'arrête et on renvoie False
- Si on trouve un état solution à une profondeur inférieure ou égale à profmax, on construit la liste des états nécessaires pour atteindre cette solution

**Pseudo code :**
```
DLS(node, goal, depth) {
  if ( depth >= 0 ) {
    if ( node == goal )
      return node

    for each child in expand(node)
      DLS(child, goal, depth-1)        //Cet algorithme effectue une remontée depuis la profondeur max jusqu'à l'état initial
  }
}
```

```
//Autre version (voir IDS)
function DLS(node, depth) is
    if depth = 0 then
        if node is a goal then
            return (node, true)
        else
            return (null, true)    (Not found, but may have children)

    else if depth > 0 then
        any_remaining ← false
        foreach child of node do
            found, remaining ← DLS(child, depth−1)
            if found ≠ null then
                return (found, true)   
            if remaining then
                any_remaining ← true    (At least one node found at depth)
        return (null, any_remaining)
```

**Code python**
```python
def dls(profmax, etatdepart, profdepart, chemin):
    """
    Fonction qui teste les états suivants à partir d'un état en profondeur tout en se limitant à une profondeur max donnée
    Entrées :
        - profmax : profondeur maximale
        - etatdepart : état de départ
        - profdepart : profondeur initiale
        - chemin : liste des états successifs d'accès à l'état gagnant
    Sorties : 
    """
    
    # vérification si la profondeur maximum n'est pas atteinte
    if profdepart > profmax:
        return(False)
    # vérification si l'etat testé est gagnant ou non
    taquindetravail = Taquin(etatdepart)
    if taquindetravail.estGagnant():
        return True
        
    # parcours récursif des états suivants
    # appel récursif de la fonction de test pour descendre en profondeurfor k in listeEtatsSuivants:
    for etatsuivant in taquindetravail.suivants():
        if dls(profmax, etatsuivant, profdepart + 1, chemin) == True:
            #stockage du chemin d'accès en mémoire et retour
            chemin.append(k)
            return True
        
    return False

profsol = bfs(taquindejeu.etat)
dls(profsol, taquindejeu.etat, 0, chemin)
```

[**Retour au sommaire**](#sommaire)

## IV.B Preuves <a id="dls-2"></a>

### Ressources 
- <a href="https://cours-examens.org/images/Etudes_superieures/ingeniorat_electronique/3_annee/Design/Design_III/algorithmes_exploration_h2011.pdf">DLS n'est pas optimal slide 31/58</a>

### IV.B.1 Terminaison

La terminaison de l'algorithme DLS découle de celle du PEP sachant que la profondeur est limitée (finie).

### IV.B.2 Correction

La correction de l'algorithme DLS est garantie par celle du PEP (qui permet de parcourir tous les états du graphe) mais il est possible qu'il ne trouve pas d'état solution selon la profondeur maximale fixée

Dans notre cas, cette profondeur maximale est déterminée par BFS au préalable, ce qui assure que l'algorithme DLS est correct et trouve un état solution.

### IV.B.3 Optimalité

En général, la profondeur maximale est choisie arbitrairement et l'algorithme DLS n'est pas optimal 
- En effet supposons que la solution optimale soit à une profondeur 10 et se trouve dans un successeurs &laquo; droit &raquo; de l'état initial
- Si de plus, on choisit la profondeur maximale à 20, comme l'algorithme DLS effectue un PEP, il peut trouver un état solutions à une profondeur > 10 qui se trouverait dans un successeur &laquo; gauche &raquo; de l'état initial
    
Dans notre cas, en supposant que BFS lui fournisse la profondeur optimale ($l=d$ d'un état solution, l'algorithme DLS sera optimal puisqu'il va parcourir tous les états à cette profondeur. 

**Take home message :**

L'optimalité de DLS est héritée dans notre cas de celle l'algorithme BFS

[**Retour au sommaire**](#sommaire)

## IV.C Complexité <a id="dls-3"></a>
- Complexité en temps : combien d'états il faut produire pour atteindre l'état solution
- Complexité en espace : nombre maximum d'états à conserver pour trouver l'état solution
- Complexité expérimentale : mesure du temps que le programme met pour se terminer

### IV.C.3 Complexité en Temps

Soit $V$ l'ensemble des états et $E$ l'ensemble des arcs du graphe d'état du taquin.<br>
Alors, avec $|V|$ est le nombre d'état du graphe et $|E|$ le nombre d'arcs : 

**Preuve :**
- L'algorithme DLS effectue un parcours en profondeur
- La complexité en temps d'un parcours en profondeur est en $O(|V|+|E|)$

**Estimation de la complexité :**
- On note $l$ la profondeur à laquelle DLS est autorisé à descendre
- L'algorithme DLS possède la même complexité en temps que l'algorithme DFS (exponentielle en la profondeur)
- Donc la complexité en espace de l'algorithme DLS est en $O(b^l)$

### IV.C.3 Complexité en Espace
- L'algorithme DLS possède la même complexité en espace que l'algorithme DFS (linéaire en la profondeur)
- Donc la complexité en espace de l'algorithme DLS est  en $O(b \times l)$

### IV.C.3 Complexité expérimentale
- **Protocole de Test**
    - Spécifications de la machine de test : 
        - Processeur Intel Core i7-8550U CPU @ 1,8GHz 
        - Mémoire RAM 8Go  
    - Lancer 100 fois l'algorithme DLS et mesurer le temps d'exécution avec le module time
    - Pour des profondeurs plus importantes, on réduira la quantité de fois où on lance DLS
    - La liste des taquins de départ utilisée est disponible dans un document annexe <br><br>

- **Code python :**

```python
import taquin, time

# Test 100 DLS
def testDLS(state, road, n):
    moy = 0
    for i in range(n):
        debut = time.time()
        proftarget = bfs(state)
        dls(proftarget, state, 0, road)
        fin = time.time() - debut
        moy += fin
    return moy/n

taquindetest = Taquin("631752048")
etat = taquindetest.etat
chemin = []
print("moyenne 100 DLS", testDLS(etat, chemin, 100))
```

- **Tableau récapitulatif :**

| Profondeur | Temps d'exécution (ordre de grandeur) |
|:---:|:---:|
|2|0,06 ms|
|3|0,20 ms|
|4|0,26 ms|
|5|1,3 ms|
|6|1,1 ms|
|7|9,6 à 15,5 ms|
|8|9,25 à 61,5 ms|
|9|48,5 à 145 ms|
|10|75 à 325 ms|
|11|400 à 1100 ms|
|12|600 à 900 ms|
|13|8 à 9 secondes|
|14|9 à 18 secondes|
|15|t > 80 s ou plante|
|16 et plus|plante|

[**Retour au sommaire**](#sommaire)

# V. IDS (Iterative Deepening Search) <a id="ids"></a>

Aussi connu sous les dénominations suivantes :
- IDDFS (Iterative Deepening Depth First Search)
- DFID (Depth First Iterative Deepening)
- En français : parcours en profondeur à approfondissement progressif)

### Ressources
- <a href="https://en.wikipedia.org/wiki/Iterative_deepening_depth-first_search">Wikipedia ENG</a>
- <a href="https://www.geeksforgeeks.org/iterative-deepening-searchids-iterative-deepening-depth-first-searchiddfs/">Une fiche sur IDS</a>
- <a href="https://www.youtube.com/watch?v=7QcoJjSVT38">Autre vidéo partie 1</a>
- <a href="https://www.youtube.com/watch?v=5MpT0EcOIyM">Autre vidéo partie 2</a>
- <a href="http://jul.andre.free.fr/Cavaliers/cours2_RechAveugle.pdf">IDS dans un cours de l'université de Nancy (à partir du slide 77/97)</a>
- <a href="">KORF : Extrait à lire absolument ! (IDS page 4 et 5/13)</a>

### V.A Principe <a id="ids-1"></a>

L'algorithme DLS travaille jusqu'à une profondeur limite $l$ (fixée).<br>
L'algorithme IDS effectue des itérations sur DLS avec une profondeur limite qui s'incrémente à chaque nouvelle itération (on descend de plus en plus profondément)

Il est vrai que des états seront alors visités plusieurs fois à chaque nouvelle itération et que cela pourrait se révéler coûteux. En réalité, on trouve davantage d'états à mesure que l'on descend en profondeur : on peut donc considérer négligeable le fait de revisiter les états se trouvant à des niveaux peu profonds [voir aussi l'étude de la complexité](#ids-3).

En réalisant ces itérations, on réalise en fait un PEL du graphe <a href="https://www.youtube.com/watch?v=kNq6oRhOT1k">(voir cette vidéo vers 2min20s)</a> 

**Pseudo code :**
```
function IDDFS(root) is
    for depth from 0 to ∞ do                           
        found, remaining ← DLS(root, depth)
        if found ≠ null then
            return found
        else if not remaining then
            return null

function DLS(node, depth) is
    if depth = 0 then
        if node is a goal then
            return (node, true)
        else
            return (null, true)    (Not found, but may have children)

    else if depth > 0 then
        any_remaining ← false
        foreach child of node do
            found, remaining ← DLS(child, depth−1)
            if found ≠ null then
                return (found, true)   
            if remaining then
                any_remaining ← true    (At least one node found at depth, let IDDFS deepen)
        return (null, any_remaining)
```

**Code python :**
```python
def ids(etattest, path):
    profmax = 0
    profinit = 0
    while not dls(profmax, etattest, profinit, path):
        profmax += 1
```

[**Retour au sommaire**](#sommaire)

## V.B Preuves <a id="ids-2"></a>

### V.B.1 Terminaison
- Dans le pseudo code, on voit que les itérations pourraient se faire jusqu'à l'infini ( for depth from 0 to ∞ do ).
- Le cas de figure correspondant serait un graphe infini ne possédant pas d'état solution du taquin. Or, le pré-conditionnement sur l'état initial du taquin nous évite ce cas de figure.
- D'où le nombre d'itérations est majoré
- La terminaison est assurée par celle de l'algorithme DLS (qui est appelé à chaque passage de la boucle for)

### V.B.2 Correction

La correction de l'algorithme IDS découle de celle de DLS, elle même héritée de celle de BFS (puisqu'en réalité c'est un PEL qui est effectué).

### V.B.3 Optimalité

IDS est optimal à cause de la progressivité sur la profondeur de la recherche (cette dernière s'exécute comme un PEL)

[**Retour au sommaire**](#sommaire)

## V.C Complexité <a id="ids-3"></a>
- Complexité en temps : combien d'états il faut produire pour atteindre l'état solution
- Complexité en espace : nombre maximum d'états à conserver pour trouver l'état solution
- Complexité expérimentale : mesure du temps que le programme met pour se terminer

### Ressources
- <a href="http://cseweb.ucsd.edu/~elkan/130/itdeep.html">Voir le bas de cette page</a>

### V.C.1 Complexité en Temps

- De part la démarche **itérative**, les états sont générés plusieurs fois, mais cela reste bien acceptable si le facteur de branchement $b$ reste similaire à chaque niveau de profondeur.
- La majorité des états se trouve ainsi au niveau le plus profond (on le note $k$)
- Les états de profondeur $k$ sont générés une fois, les états de profondeur $(k-1)$ sont générés $2$ fois, et ainsi de suite en remontant jusqu'à l'état de départ du taquin
- Ainsi, le nombre d'états générés dans le pire cas est $k \times b + (k-1) \times b^2 + ... + 2 \times b^{k-1} + 1 \times b^k = \displaystyle \sum_{i=1}^{k}i \cdot b^{1-i}$
- Ce nombre d'états est $\leq b^k \displaystyle \sum_{i=1}^{\infty}i \cdot b^{1-i} = b^k \cdot \left( \dfrac{b}{b-1} \right)^2$
- Cela va poser des problèmes si le facteur de branchement $b=1$ mais on a vu que dans notre cas, en moyenne, $b \simeq 1,66$
- D'où la complexité en temps est en $O(b^k)$
- Comme l'algorithme IDS est optimal, sa complexité en temps est en $O(b^d)$ (on reprend la notation $d$ de la profondeur de l'état solution de BFS)

**Take home message :**
- Le complexité en temps de l'algorithme IDS est similaire à celle de BFS (exponentielle en la profondeur)

### V.C.2 Complexité en Espace

- Si le parcours effectué par l'algorithme IDS s'effectue comme un PEL, la stratégie concernant la gestion de la mémoire est celle d'un PEP
- En effet, à tout moment, IDS est engagé dans un PEP via l'appel à DLS
- D'où la complexité en espace de l'algorithme est similaire à celle de l'algorithme DLS 
- Donc la complexité en espace de l'algorithme IDS est en $O(b \times k)$
- Comme l'algorithme IDS est optimal, sa complexité en espace est en $O(b \times d)$ (on reprend la notation $d$ de la profondeur de l'état solution de BFS)

**Moralité :**
- L'algorithme IDS combine le meilleur de BFS (optimalité) et de DFS (espace mémoire réduit)
- L'algorithme IDS est une méthode à employer lorsque l'espace de recherche est vaste et que la profondeur de la solution n'est pas connue

### V.C.3 Complexité expérimentale

- **Protocole de Test**
    - Spécifications de la machine de test : 
        - Processeur Intel Core i7-8550U CPU @ 1,8GHz 
        - Mémoire RAM 8Go  
    - Lancer 100 fois l'algorithme IDS et mesurer le temps d'exécution avec le module time
    - Pour des profondeurs plus importantes, on réduira la quantité de fois où on lance IDS
    - La liste des taquins de départ utilisée est disponible dans un document annexe <br><br>

- **Code python :**

```python
import taquin, time

# Test 100 IDS
def testIDS(state, road, n):
    moy = 0
    for i in range(n):
        debut = time.time()
        ids(state, road)
        fin = time.time() - debut
        moy += fin
    return moy/n

taquindetest = Taquin("631752048")
etat = taquindetest.etat
chemin = []
print("moyenne 100 IDS", testIDS(etat, chemin, 100))
```

- **Tableau récapitulatif :**

| Profondeur | Temps d'exécution (ordre de grandeur) |
|:---:|:---:|
|2|0,06 ms|
|3|0,20 ms|
|4|0,26 ms|
|5|1,3 ms|
|6|1,1 ms|
|7|9,6 à 15,5 ms|
|8|9,25 à 61,5 ms|
|9|48,5 à 145 ms|
|10|75 à 325 ms|
|11|400 à 1100 ms|
|12|600 à 900 ms|
|13|8 à 9 secondes|
|14|9 à 18 secondes|
|15|t > 80 s ou plante|
|16 et plus|plante|

**Take home message :**
- Pour des profondeurs faibles, l'algorithme IDS est légèrement plus efficace que DLS
- Pour des profondeurs plus importantes, le fait de re-générer des états déjà visités encombre vraiment la vitesse d'exécution

[**Retour au sommaire**](#sommaire)

# VI. IDA* (Iterative Deepening A*) <a id="ida"></a>

### Ressources générales

- <a href="">KORF : Extrait à lire absolument ! (IDA* à partir de la page 6/13)</a>
- <a href="http://archive.stock.free.fr/memoire/Farreny.pdf">Un peu d'histoire sur A* et IDA*</a>
- <a href="https://perso.liris.cnrs.fr/christine.solnon/tp2.pdf">Un exemple de TP avec A* (Christine SOLNON)</a>
- <a href="http://www.cril.univ-artois.fr/~benferhat/Projet_TP_3.pdf">Un autre exemple de TP/Projet (Université d'Artois - Hauts de France)</a>

### Ressources Rappels

#### Rappel 1 : A*
- <a href="https://fr.wikipedia.org/wiki/Heuristique_(math%C3%A9matiques)">Wikipedia FR</a>.
- <a href="https://perso.liris.cnrs.fr/pierre-edouard.portier/teaching_2014_2015/ia/astar/astar.html#sec-5-2">Lien entre Dijkstra et A*</a>
- <a href="https://www.youtube.com/watch?v=vP5TkF0xJgI">Une présentation en anglais de A*</a>
- <a href="https://fr.slideshare.net/mohamedhenyselmi/intelligence-artificielle-algorithmes-de-recherche">Recherche informée à base d'heuristique slides 68 à 71/109</a>

#### Rappel 2 : Admissibilité d'une heuristique
- <a href="https://fr.wikipedia.org/wiki/Algorithme_A*">Wikipedia FR</a>
- <a href="http://p6e7p7.freeshell.org/teaching_2017_2018/ia/heuristics/">Une heuristique cohérente est admissible</a>
- <a href="http://informatique.umons.ac.be/algo/astar.pdf">Des slides en français (voir slide 69/75 à 71/75 pour des exemples d'heuristiques pour le taquin)</a>

## VI.A Principe <a id="ida-1"></a>

On combine la stratégie ID (Iterative Deepening ou approfondissement progressif) précédente avec la stratégie de l'algorithme A* qui consiste à utiliser une (fonction) &laquo; heuristique &raquo;. Il s'agit donc d'un algorithme de recherche réalisant un PEP et dont la profondeur limite est déterminée par une fonction.<br>
De plus, la profondeur limite est modifiée à chaque itération : on la fixe en estimant le nombre de coups minimum restant à faire pour atteindre l'état solution depuis chaque état testé.

**Rappel 1 : Algorithme A* et heuristiques**

L'algorithme **A*** est une amélioration de l'algorithme de **Dijkstra** grâce à l'emploi d'une (fonction) &laquo; heuristique &raquo;.<br>
Une heuristique est une méthode de calcul qui fournit rapidement une solution réalisable, pas nécessairement optimale ou exacte, pour un problème d'optimisation difficile 

L'algorithme **A*** associe à un état $u$ du graphe la valeur : $f(u) = g(u) + h(u)$ où $g(u)$ est le poids optimal actuellement connu pour un chemin menant de $s$ à $u$, et $h(u)$ est une heuristique &laquo; admissible &raquo;  (i.e., une borne inférieure de la distance séparant $u$ de la cible $t$).

**Rappel 2 : Admissibilité d'une heurisitique**

Un algorithme de recherche qui garantit de toujours trouver le chemin le plus court à un but s'appelle &laquo; algorithme admissible &raquo;. <br>
Une heuristique qui rend un algorithme admissible est elle-même appelée &laquo; admissible &raquo;.<br>
Ainsi, pour A* , s'il utilise une heuristique qui ne surestime jamais la distance (ou plus généralement le coût) du but, **A*** est considéré comme admissible. 

**Take home message :**
- Une heuristique admissible ne surestime jamais la &laquo; vraie &raquo; distance entre l'état initial et l'état solution
- Pour aller plus loin, on peut montrer qu'il y a <a href="http://p6e7p7.freeshell.org/teaching_2017_2018/ia/heuristics/">équivalence entre heuristique cohérente et heuristique monotone</a> et qu'une heurisitique cohérente est admissible (au sens de l'implication)
- Attention l'implication réciproque est fausse

### Choix de l'heuristique 

Il existe plusieurs heuristiques et nous devons en choisir une qui nous permet d’évaluer rapidement le nombre de coups nécessaires pour atteindre la solution sans surévaluer ce nombre. <br>
On peut envisager d’utiliser la mesure de dérangement présentée au [I.C Chaos](#chaos) mais il existe une autre façon de faire qui est plus efficace (voir <a href="https://www.irif.fr/~kesner/enseignement/iup/cours61.pdf">heuristique dominante (slide 49/52)</a>)

On part du principe que l'état solution est la configuration du taquin suivante : <img src="images/manhattan0.png" style="width: 15%">
Alors toute modification de ce taquin demande un certain nombre de coups pour y parvenir. <br>
<img src="images/manhattan1.png" style="width: 90%">
Le principe de l’heuristique que nous allons utiliser est de déterminer le nombre de coups minimum pour ramener chaque tuile à sa position initiale sans s’occuper des autres.

Pour reprendre nos exemples :<img src="images/manhattan2.png" style="width: 70%">

Et pour le dernier : 
<img src="images/manhattan3.png" style="width: 25%">
Soit un total d’au moins 12 coups rien que pour replacer chaque tuile sur sa position initiale. <br>
Expérimentalement, pour ranger le taquin il faut aussi comptabiliser les mouvements engendrés pour les autres tuiles par ces 12 coups, ce qui donne un total de 20 coups. <br>
C'est cependant la valeur 12 qui sera retenue pour l'évaluation de l'heuristique à chaque itération car si à un état donné, il faut plus de 12 coups, c'est que les successeurs n'engendrent pas une zone de recherche &laquo; intéressante &raquo;.

### Normalisation

Ce que nous venons de calculer ici est la somme de la distance de Manhattan de chaque tuile. 

**Définition :** La distance de Manhattan, appelée aussi &laquo; taxi-distance &raquo;, est la distance parcourue par un taxi lorsqu'il se déplace entre deux points dans une ville où les rues sont agencées selon un réseau ou quadrillage (comme à Manhattan, quartier de New York aux USA d'où le nom). Un taxi-chemin est le trajet fait par un taxi lorsqu'il se déplace d'un nœud du réseau à un autre en utilisant les déplacements horizontaux et verticaux du réseau.
<img src="images/manhattan4.png" style="width: 20%">

**Propriété :**
On montre que si l'on oriente le réseau et que l'on définit des déplacements élémentaires positifs et négatifs, la distance de Manhattan est indépendante du chemin parcouru à l'intérieur d'un réseau fini. Ainsi, sur l'image ci dessus, la distance entre les deux points noirs, qu'on les joigne par les chemins rouge, bleu ou jaune, est identique.

**Take home message :**
- Entre deux points $A$ et $B$, de coordonnées respectives $( X_A , Y_A )$ et $( X_B , Y_B )$, la distance de Manhattan est définie par : $d(A,B)=|X_B − X_A| + |Y_B − Y_A|$

### Application au taquin 3X3 

- On modélise le taquin par la matrice suivante :<img src="images/manhattan5.png" style="width: 20%">
On repère une case de la grille par sa ligne $i$ (avec $0 \leq i \leq 2$ de haut en bas) et sa colonne $j$ (avec $0 \leq j \leq 2$ de gauche à droite). <br>
Si $e$ est un état du taquin et $v$ un entier entre 1 et 8 on note $e^i_v$ la ligne de l’entier $v$ dans $e$ et $e^j_v$ la colonne de l’entier $v$ dans $e$.  

- Le nombre de coups associé à l’élément $v$ est donc égal à la somme :
    - de l’écart entre sa ligne actuelle et sa ligne d’origine 
    - et de l’écart entre sa colonne actuelle et sa colonne d’origine
    - c'est-à-dire $|e^i_v - E[v/3] |+| e^j_v - (v \hspace{3pt} mod \hspace{3pt} 3) |$ où $E[.]$ désigne la partie entière et $mod$ le reste modulo

- C'est-à-dire, suivant les valeurs de $v$ :

|   $v$   | Partie entière $E[v/3]$ | $v$ Modulo $3$ |
|:-------:|------------------------:|---------------:|
| **0** | 0 | 0 | 0 |
| **1** | 0 | 1 | 0 |
| **2** | 0 | 2 | 0 |
| **3** | 1 | 0 | 0 |
| **4** | 1 | 1 | 0 |
| **5** | 1 | 2 | 0 |
| **6** | 2 | 0 | 0 |
| **7** | 2 | 1 | 0 |
| **8** | 2 | 2 | 0 |

- Ce qui donne suivant les valeurs de $v$ :

<img src="images/manhattan3.png" style="width: 25%">

|   $v$   | Ecart abscisses | Ecart ordonnées | Taxi-distance |
|:-------:|----------------:|:---------------:|:-------------:|
| **0** | 0 | 0 | 0 |
| **1** | 1 | 2 | 3 |
| **2** | 0 | 1 | 1 |
| **3** | 1 | 0 | 1 |
| **4** | 0 | 1 | 1 |
| **5** | 2 | 1 | 3 |
| **6** | 0 | 1 | 1 |
| **7** | 0 | 0 | 0 |
| **8** | 0 | 2 | 2 |
| **Total** | 4 | 8 | **12** |

- Dans ces conditions on définit la fonction $h$ pour un état $e$ du taquin de la façon suivante : $h(e)=\displaystyle \sum_{v=0}^{8} |e^i_v - E[v/3] | + | e^j_v - (v \hspace{3pt} mod \hspace{3pt} 3)|$

**Admissibilité** : 
- On montre que cette fonction $h$ est monotone (décroissante) puisque cohérente $\Leftrightarrow$ monotone et cohérente $\Rightarrow$ admissible
- Si $e_s$ est l'état solution, chaque case est exactement à sa position initiale, et on a $h(e_s)=O$ car pour chaque $v$, $e^i_v = E[v/3]$ et $e^j_v = v \hspace{3pt} mod \hspace{3pt} 3$
- Par ailleurs, tout déplacement du taquin ne peut réduire au maximum que de 1 la valeur de $h$ (en modifiant soit une valeur de $e^i_v$ soit une valeur de $e^j_v$ de 1)
- Ainsi, pour une configuration donnée $e$, il faut bien au moins $h(e)$ coups pour arriver à annuler la fonction $h$, donc pour arriver à l’état solution $e_s$
- La fonction $h$ donne donc bien un minorant du nombre de coups pour atteindre la solution et elle est donc admissible comme heuristique

### Utilisation la fonction heuristique pour améliorer les performances des algorithmes de recherche

La fonction heuristique que nous venons de proposer nous permet donc d'évaluer, tout en lui restant inférieur, le nombre de coups nécessaires pour parvenir d'un état donné à la solution.

Ceci nous permet de réduire le nombre de branches à tester lors de la recherche.

Pour chaque état testé, on ne va pas seulement vérifier si l'état est gagnant ou non mais également comparer la grandeur $C(e)=p(e)+h(e)$ à la profondeur maximale $Pmax$ de la solution (avec $p(e)$ la profondeur de l'état testé et $h(e)$ le calcul de la fonction heuristique pour l'état $e$) :
- soit l'état est gagnant et la recherche est terminée
- soit $C(e)\ge Pmax$ et la branche n'est donc plus à explorer
- soit $C(e)\le Pmax$ et on continue la recherche

Ceci est faisable car $h(e)$ renvoi bien un minorant du nombre de coups et donc $C(e)$ est lui même un minorant du nombre d'états à tester.

On peut observer dans l'image ci-dessous [<a href="https://www.geeksforgeeks.org/8-puzzle-problem-using-branch-and-bound/">Source (voir schéma après le pseudo code)</a>] un exemple pour une solution en profondeur 5 :
<img src="images/nbcoup.png" style="width: 40%">


**Code python :**
```python
# Fonction d'heuristique
def nbcoup(etat):
    total = 0
    rang = -1
    coord = [(0,0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)]
    
    for i in etat:
        rang += 1
        
        if i != '0':
            val = int(i)
            # écart de la case par rapport aux lignes
            terme1 = int(val/3)
            # écart de la case par rapport aux colonnes
            terme2 = (fmod(val,3))
            
            part1 = fabs(coord[rang][0] - terme1)
            part2 = fabs(coord[rang][1] - terme2)
            total += part1 + part2
    
    return(total)

# Version de DLS intégrant l'heuristique
def dlsplus(profmax, etatdepart, profdepart, chemin):
	#fonction qui test les suivants d'un etat en profondeur tout en se limitant a une profondeur donnée
    global min
    taquindetravail = Taquin(etattest)
    
    #évalue le nombre de coup minimum pour atteindre la solution en ajoutant le minorant du nombre de coups à la profondeur actuellement testée
    evalcoup = profdepart + nbcoup(etatdepart)
    #print('nb coup a la solution (evalcoup)=',evalcoup)
        
    # vérification si la profondeur maximum n'est pas atteinte en tenant compte de la minoration du nombre de coup
    if evalcoup > profmax:
        #print('nb profmax depassé')
        #print('min=',min,' evalcoup=',evalcoup)
        if evalcoup < min:
            min = evalcoup
        return False
    
    # verification si l'etat testé est gagnant ou non
    if taquindetravail.estGagnant():
        return True
    
    # Appel récursif de la fonction pour descendre en profondeur
    for etatsuivant in taquindetravail.suivants():
        if dlsplus(profmax, etatsuivant, profdepart + 1, chemin) == True:
            #stockage du chemin d'accès en mémoire et retour
            chemin.append(k)
            return True
    
    return(False)

# Recherche avec IDA*
def ida(etatdepart, chemin):
    global min
    
    # Evaluation à l'aide de l'heuristique de la profondeur à laquelle descendre
    m = nbcoup(etatdepart)
    
    while m != 1000:
        min = 1000
        if dlsplus(m, etatdepart, 0, chemin):
            #print('m=',m,' min=',min)
            return(True)
        #print('maj de m')
        m = min
    
    return False
```

[**Retour au sommaire**](#sommaire)

## VI.B Preuves <a id="ida-2"></a>

### Ressources
- <a href="http://turing.cs.pub.ro/auf2/html/chapters/chapter3/chapter_3_2_1.html">Voir le bas de la page</a>
- <a href="http://p6e7p7.freeshell.org/teaching_2017_2018/ia/heuristics/">Un corrigé d'optimalité de A*</a>
- <a href="https://www.irif.fr/~kesner/enseignement/iup/cours61.pdf">A* est optimal preuve slide 45/52</a>

### VI.B.1 Terminaison

- L'algorithme IDA* réalise un parcours en profondeur itérée
- Sa terminaison est donc héritée de celle de l'algorithme IDS

### VI.B.2 Correction

- L'algorithme IDA* réalise un parcours en profondeur itérée
- Sa correction est donc héritée de celle de l'algorithme IDS

### VI.B.3 Optimalité

L'optimalité de l'algorithme IDA* est garantie par l'admissibilité de l'heuristique choisie

[**Retour au sommaire**](#sommaire)

## VI.C Complexité <a id="ida-3"></a>

- Complexité en temps : combien d'états il faut produire pour atteindre l'état solution
- Complexité en espace : nombre maximum d'états à conserver pour trouver l'état solution
- Complexité expérimentale : mesure du temps que le programme met pour se terminer

### Ressources

- <a href="https://ai.stackexchange.com/questions/8821/how-is-iterative-deepening-a-better-than-a">Stack exchange forum</a>
- <a href="https://moodle.insa-rouen.fr/pluginfile.php/39139/mod_resource/content/0/Res-PB.pdf">Complexité pour A* slide 18/30</a>
- <a href="https://www.sciencedirect.com/science/article/pii/S0004370201000947/pdf?md5=6685f651efae7e0f243de5d7d4522aba&pid=1-s2.0-S0004370201000947-main.pdf">KORF à lire absolument (à partir de la page 12/20)</a>

### VI.C.1 Complexité en Temps

- On reprend l'évaluation de la complexité pour l'algorithme IDS : le principe itératif montre que les états sont générés plusieurs fois
- Dans le pire cas avec l'algorithme IDS, le nombre d'états générés est $k \times b + (k-1) \times b^2 + ... + 2 \times b^{k-1} + 1 \times b^k = \displaystyle \sum_{i=1}^{k}i \cdot b^{1-i}$, ce qui donnait une complexité en temps pour l'algorithme IDS en $O(b^d)$
- Dans le cas de l'algorithme IDA*, on ne génère pas tous les états du niveau de profondeur suivant puisqu'on évalue via la fonction heuristique le coup des états suivants
- Le nombre d'états générés est donc $k \times b^* + (k-1) \times (b^*)^2 + ... + 1 \times (b^*)^d$, où $b^*$ est le facteur de branchement modifié par l'heuristique et qui est meilleur que le facteur de branchement habituel $b$
- Ce qui donne une complexité en temps pour l'algorithme IDA* en $O((b^*)^d)$

**Evaluation de la complexité :**
- Dans un arbre de recherche, s'il n'y a qu'un seul état solution et que la fonction heuristique $h$ vérifie la condition $|h(x) - h^*(x)| = O(\log h^*(x))$, où $h^*$ est le coût optimal alors la complexité en temps de l'algorithme de recherche devient polynomiale
- En pratique sur le taquin, $b^*$ est très proche de $1$
- La complexité qui semblait exponentielle se réduit à une complexité polynomiale (voire linéaire si $b^* = 1$)

**Take home message :**
- En général la complexité en temps de l'algorithme IDA* est en $O(b^d)$ (pire cas)
- En pratique et suivant l'état initial, la complexité en temps de l'algorithme IDA* s'approche d'une complexité polynomiale
- C'est pourquoi des écarts importants peuvent être observés dans les résultats expérimentaux

### VI.C.2 Complexité en Espace

- L'algorithme IDA* :
    - s'appuie sur un parcours en profondeur itérée
    - est guidé par une stratégie (heuristique)
- D'où la complexité en espace de l'algorithme IDA* est similaire à celle de l'algorithme IDS
- Donc la complexité en espace de l'algorithme IDA* est en $O(b \times d)$

**Take home message :**
- Généralement, avec l'algorithme A*, on manque d'espace bien avant de manquer de temps
- L'algorithme IDA* a nécessite moins d'espace mémoire que l'algorithme A* (qui garde tous les états visités en mémoire)

### VI.C.3 Complexité expérimentale

- **Protocole de Test**
    - Spécifications de la machine de test : 
        - Processeur Intel Core i7-8550U CPU @ 1,8GHz 
        - Mémoire RAM 8Go  
    - Lancer 100 fois l'algorithme IDA* et mesurer le temps d'exécution avec le module time
    - Pour les valeurs les plus hautes de profondeur, on réduira la quantité de fois où on lance IDA*
    - La liste des taquins de départ utilisée est disponible dans un document annexe <br><br>
  
- **Code python :**

```python
import taquin, time

# Test 100 IDA*
def testIDA(state, road, n):
    moy = 0
    for i in range(n):
        debut = time.time()
        ida(state, road)
        fin = time.time() - debut
        moy += fin
    return moy/n

taquindetest = Taquin("631752048")
etat = taquindetest.etat
chemin = []
print("moyenne 100 IDA", testIDA(etat, chemin, 100))
```

- **Tableau récapitulatif :**

| Profondeur | Temps d'exécution (ordre de grandeur) |
|:---:|:---:|
|2|0,18 ms|
|3|0,20 ms|
|4|0,22 ms|
|5|0,23 ms|
|6|0,24 ms|
|7|0,28 ms|
|8|0,29 à 0,37 ms|
|9|0,32 à 1 ms|
|10|0,32 à 1 ms|
|11|0,35 à 12 ms|
|12|t < 1 ms|
|13|t < 15 ms|
|14|t < 19 ms|
|15|t < 2 ms|
|16|t < 2 ms|
|17|t < 2 ms|
|18|t < 1 s|
|19|t < 4 s|
|20|t < 5 s|
|21|t < 3 s|
|23|t < 35 s|
|24|t < 5 s|
|25|t < 15 s|
|26|t < 2 min|
|27|t < 1 min|

**Take home message :**
- Le temps d'exécution reste &laquo; acceptable &raquo; même aux grandes profondeurs
- <a href="http://mpechaud.fr/scripts/parcours/testtaquin.html">Le site de M. Pechaud pour faire des tests</a>

[**Retour au sommaire**](#sommaire)

## VII. TLDR (Too Long Didn't Read : En résumé) <a id="tldr"></a>

### Ressources

- <a href="https://books.google.fr/books?id=ekUHwvRP7nUC&pg=PA47&lpg=PA47&dq=is+DLS+%3D+BFS+%2B+DFS&source=bl&ots=ofbvjKatcL&sig=ACfU3U2TnZtDhDXQt7YucpFqXgjFETL3dA&hl=fr&sa=X&ved=2ahUKEwiChoKkgdzoAhUYpp4KHeoFDpoQ6AEwEnoECAoQKw#v=onepage&q=IDA*&f=false">Un livre à lire (anglais)</a>
- <a href="https://mhesham.wordpress.com/tag/depth-limited-search/">Synthèse BFS, DLS et IDS</a>
- <a href="https://courses.cs.washington.edu/courses/cse473/12au/slides/lect3.pdf">Un cours complet en anglais</a>
- <a href="https://www.lri.fr/~antoine/Courses/AGRO/Cours-IA/Tr-IA-graphes-2007x4.pdf">Un cours complet avec quelques comparaisons à la fin entre IDS et A*</a>
- <a href="https://www.sciencedirect.com/science/article/pii/S0004370201000947/pdf?md5=6685f651efae7e0f243de5d7d4522aba&pid=1-s2.0-S0004370201000947-main.pdf">La référence sur IDA*</a>

**Une classification :**
- Algorithmes de recherche non-informée (recherche aveugle)
    - BFS
    - DFS
    - IDS
- Algorithme de recherche informée (avec heuristique)
    - A*
    - IDA*

**Des notations :**
- $G$ : graphe de recherche, supposé fini dans notre cas
- $b$ : nombre de successeurs d'un état (branching factor, vaut au plus 4 dans le taquin)
- $d$ : profondeur à laquelle se trouve l'état solution cherché
- $m$ : profondeur maximum dans le graphe (fini puisque $G$ est supposé fini dans notre cas)
- $l$ : profondeur maximum à laquelle l'algorithme est autorisé à chercher

**Deux tableaux récapitulatifs :**

|ALGORITHMES|Complétude|Optimalité|Complexité en Temps|Complexité en Espace|Cas d'utilisation|
|:---------:|:--------:|:--------:|:------------------:|:-----------------:|:---------------:|
|**BFS**|Oui si $b$ est fini|Oui si coût constant|$O(b^d)$|$O(b^d)$|Solut° proche de l'état initial|
|**DFS**|Oui si pas de cycles|Non|$O(b^m)$, horrible si $m>>d$|$O(b \times m)$|Solut° denses (nombreuses)|
|**DLS**|Non si $l<d$|Non|$O(b^l)$|$O(b \times l)$|Connaissance à priori du problème (sur $l$)|
|**IDS**|Oui si $b$ est fini|Oui si coût constant|$O(b^d)$|$O(b \times d)$|Profondeur solut° inconnue|
|**A***|Oui|Oui si heuristique consistante|$O(b^d)$ mais voir heurisitique|$O(b^d)$|Recherche du plus court chemin|
|**IDA***|Oui|Oui si heuristique consistante|$O(b^d)$ mais voir heurisitique|$O(b \times d)$|Plus court chemin \& optimisat° mémoire|

|        CRITERES        | BFS | DFS | DLS | IDS | A\* | IDA\* |
|:----------------------:|:---:|:---:|:---:|:---:|:---:|:-----:|
|**Complétude**          |Oui si $b$ est fini|Oui si pas de cycles|Non si $l<d$|Oui si $b$ est fini|Oui|Oui|
|**Complexité en Temps** |$O(b^d)$|O($b^m$), horrible si $m>>d$|$O(b^l$)|$O(b^d$)|$O(b^d)$ mais voir heurisitique|$O(b^d)$ mais voir heuristique|
|**Complexité en Espace**|$O(b^d)$|$O(b \times m$)|$O(b \times l$)|$O(b \times d$)|$O(b^d)$|$O(b \times d$)|
|**Optimalité**          |Oui si coût constant|Non|Non|Oui|Oui si heuristique consistante|Oui si heuristique consistante|
|**Cas d'utilisation**   |Solut° proche de l'état initial|Solut° denses (nombreuses)|Connaissance à priori du problème (sur $l$)|Profondeur solut° inconnue|Recherche du plus court chemin|Plus court chemin \& optimisat° mémoire|

**D'autres algorithmes possibles :**
- UCS (Uniform Cost Search)
- BFS (Best First Search, à ne pas confondre avec BFS pour Breadth First Search)
- SMA* (Simplified Memory-bound A*)
- WA* (Weighted A*)
- Backtracking

**Autres :**
- <a href="https://github.com/originecode/Taquin">Un autre projet Taquin focalisé sur A*</a>

[**Retour au sommaire**](#sommaire)

# Licence<a id="licence"></a>
DIU EIL, ressource éducative libre distribuée sous [Licence Creative Commons Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/) ![Licence Creative Commons](https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png)